### *Localisation optimale des capacités de production éoliennes en Europe*

Notebook pour le  chargement des données du projet (partie I) -- version 1.2 (màj le 31/3/2020)

In [1]:
## imports classiques
import numpy as np

from cylp.cy import CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPArray, CyLPModel

import matplotlib.pyplot as plt
import math 
import pandas as pd
import plotly.graph_objects as go
import matplotlib.colors as colors
import random


colors_list = list(colors._colors_full_map.values())
random.shuffle(colors_list)

%matplotlib notebook

In [2]:
## paramètres

liste_pays = ['Allemagne','Autriche','Belgique','Danemark','Espagne','France','Irlande','Italie','Luxembourg',\
             'Norvège','Pays-Bas','Portugal','Royaume-Uni','Suède','Suisse']

liste_pays_index_dic = {'Allemagne':0,'Autriche':1,'Belgique':2,'Danemark':3,'Espagne':4,'France':5,'Irlande':6,'Italie':7,'Luxembourg':8,\
             'Norvège':9,'Pays-Bas':10,'Portugal':11,'Royaume-Uni':12,'Suède':13,'Suisse':14}

list_colors_dic = {'Allemagne':colors_list[0],'Autriche':colors_list[1],'Belgique':colors_list[2],'Danemark':colors_list[3],'Espagne':colors_list[4],\
                         'France':colors_list[5],'Irlande':colors_list[6],'Italie':colors_list[7],'Luxembourg':colors_list[8],\
             'Norvège':colors_list[9],'Pays-Bas':colors_list[10],'Portugal':colors_list[11],'Royaume-Uni':colors_list[12],'Suède':colors_list[13],'Suisse':colors_list[14]} 

TURB = np.array([8587 , 12009 , 1417 , 9 , 18372 , 25132 , 527 , 21117 , 1140 , 28941 , 37 , 5052 , 4269 , 16637 , 15101])
POMP = np.array([5223 , 3580 , 1307 , 0 , 5347 , 4303 , 292 , 7544 , 1100 , 1396 , 0 , 1029 , 2744 , 45 , 1636 ])
RES = np.array([0.3*1e6 , 3.2*1e6 , 0.01*1e6 , 0 , 18.4*1e6 , 9.8*1e6 , 0.24*1e6 , 7.9*1e6 , 0.005*1e6 , 84.147*1e6 , 0 , 2.6*1e6 , 1.2*1e6 , 33.756*1e6 , 8.4*1e6])

N_pays = 15                  # plus petit ou égal à N_pays_max = 15
N_sites = 642                # plus petit ou égal à sites_max = 642
N_heures_par_annee = 24*365  # nombre d'heures par année
N_annees = 1/12              # fraction de l'annee à utiliser ; prendre par exemple 1/12 pour tester sur un mois (calculs 
                             # plus rapides) ; dans le rapport il faut utiliser l'année complète N_annees = 1
#N_heures = int(np.ceil(N_heures_par_annee*N_annees)) # nombre d'heures à extraire des fichiers de rendements
N_heures = 732

vecteur_temps = np.arange(1,N_heures+1) / N_heures_par_annee

In [3]:
## chargement et traitement des données brutes (I)
 
sites = pd.read_csv('Sites.csv', index_col = "index site").sort_index()

onshore = sites[sites['capacite offshore']=='Non'].copy()
offshore = sites[sites['capacite offshore']=='Oui'].copy()

In [4]:
sites.head()

,latitude,longitude,pays,couleur,capacite offshore,scores,capacites
index site,,,,,,,
0,54.0,7.0,Allemagne,#7b0323,Oui,0.006342,8364.150000
1,54.0,8.0,Allemagne,#7b0323,Non,0.003291,7509.607003
2,54.0,9.0,Allemagne,#7b0323,Non,0.002193,5003.709632
3,54.0,10.0,Allemagne,#7b0323,Non,0.001852,4225.781588
4,54.0,11.0,Allemagne,#7b0323,Non,0.001852,4225.781588


In [5]:
onshore.head()

,latitude,longitude,pays,couleur,capacite offshore,scores,capacites
index site,,,,,,,
1,54.0,8.0,Allemagne,#7b0323,Non,0.003291,7509.607003
2,54.0,9.0,Allemagne,#7b0323,Non,0.002193,5003.709632
3,54.0,10.0,Allemagne,#7b0323,Non,0.001852,4225.781588
4,54.0,11.0,Allemagne,#7b0323,Non,0.001852,4225.781588
5,54.0,12.0,Allemagne,#7b0323,Non,0.002193,5003.709632


In [6]:
offshore.head()

,latitude,longitude,pays,couleur,capacite offshore,scores,capacites
index site,,,,,,,
0,54.0,7.0,Allemagne,#7b0323,Oui,0.006342,8364.150000
48,56.0,5.0,Danemark,#A9A9A9,Oui,0.010571,15478.589744
49,55.0,6.0,Allemagne,#7b0323,Oui,0.010571,13940.250000
61,52.0,3.0,Pays-Bas,#04d8b2,Oui,0.006342,2349.264706
66,58.0,10.0,Suède,#758da3,Oui,0.006342,8577.125000


In [7]:
# chargement et traitement des données brutes (II)
rend_offshore_brut = np.genfromtxt('Rendements_offshore.csv', delimiter=',')[:,:N_heures]
rend_onshore_brut = np.genfromtxt('Rendements_onshore.csv', delimiter=',')[:,:N_heures]

print(rend_offshore_brut.shape)
print(rend_onshore_brut.shape)

(642, 732)
(642, 732)


In [8]:
indice_offshore= offshore.index
indice_onshore= onshore.index

In [9]:
rend_sites_matrix = rend_onshore_brut.copy()   # matrix combinant les valeurs réelles onshore/offshore -> souci de visibilité ci-dessous
for i in np.array(indice_offshore):
    rend_sites_matrix[i] = rend_offshore_brut[i]
#print(rend_sites_matrix)
#print(rend_sites_matrix[443][0] == rend_offshore_brut[443][0])

# Matrice 3D (nombres de sitesXnombres d'intervallesXpas) 
adpt = rend_sites_matrix.copy().reshape(N_sites, N_heures//3, 3)
#print(adpt)

# Matrice 2D (nombres de sitesXnombres d'intervalles)
sum2D = np.sum(adpt, axis = 2)
print(len(sum2D[0]))
print(sum2D)

sum2Dbis = sum2D.copy()
last = sum2Dbis[:,-1]
sum2Dbis = np.delete(sum2Dbis, np.s_[-1],  1)
sum2Dbis = np.insert(sum2Dbis,0, last, axis = 1)
print(len(sum2Dbis[0]))
print(sum2Dbis)

diff = sum2Dbis - sum2D
print(diff.shape)
a = np.array([2,3,4,5,6,7])
print(a[1:])

244
[[2.53174    2.67       2.67       ... 1.97972    1.63783    1.20162   ]
 [0.61915    1.2167     2.11155    ... 2.70167    2.60668    2.3129    ]
 [2.76648    2.80416    2.74107    ... 1.52751    1.72501    1.68017   ]
 ...
 [2.6472     1.89716    1.21242    ... 0.69577    0.362663   0.12717   ]
 [0.69489    0.142022   0.0406452  ... 0.0487501  0.095049   0.058117  ]
 [0.         0.         0.00451312 ... 0.98189    1.06166    0.86361   ]]
244
[[1.20162   2.53174   2.67      ... 2.28955   1.97972   1.63783  ]
 [2.3129    0.61915   1.2167    ... 2.735     2.70167   2.60668  ]
 [1.68017   2.76648   2.80416   ... 1.22431   1.52751   1.72501  ]
 ...
 [0.12717   2.6472    1.89716   ... 1.14926   0.69577   0.362663 ]
 [0.058117  0.69489   0.142022  ... 0.0033056 0.0487501 0.095049 ]
 [0.86361   0.        0.        ... 0.77638   0.98189   1.06166  ]]
(642, 244)
[3 4 5 6 7]


# Début du programme de résolution simplex

## Modélisation du problème d'optimisation

### Introduction des variables et paramètres


$\underline{\text{Variables}}$: 

 - $p^{installée} : \text{Vecteur des puissances intsallées sur chaque site.}$
 - $v : \text{Vecteur des variables intermédiaires permettant la retranscription des contraintes en valeur absolue.}$

$\underline{\text{Données}}$:

 - $ \eta : \text{ Matrice des rendements dans laquelle chaque ligne correspond à un site et chaque colonne à une heure}$

$\underline{\text{Paramètres généraux}}$:
    
 - $ N_{sites} : \text{ Nombre de sites sur lesquels se porte notre étude.}$
 - $ N_{heures} : \text{ Période sur laquelle se porte notre étude en heure.} $
 - $ P : \text{ Puissance totale à installer en MW.}$

$\underline{\text{Paramètres pour la variabilité}}$:

 - $ T : \text{ Pas de temps pour l'étude de la variabilité.}$
 - $ \delta  : \text{ Ratio toléré entre deux périodes de temps T consécutives.}$

$\underline{\text{Paramètres pour la répartition des sites}}$:

 - $ \kappa : \text{ Proportion de production se faisant en site offshore.}$



### Objectifs 

$
\text{max } \sum_{i=1}^{N}p_{i}^{installée}\sum_{j=1}^{N_{heures}}\eta_{ij}
$

Pour nous adapter aux spécifications du module CyLP nous avons transformé notre max en min comme suit:

$
\text{min } \sum_{i=1}^{N}p_{i}^{installée}(-\sum_{j=1}^{N_{heures}}\eta_{ij})
$

### Contraintes

 - $ \sum_{i=1}^{N} p_{i}^{installée} = P $

 - $
\sum_{i=1}^{N} \alpha_{i} p_{i}^{installée} = \kappa P 
\text{ , avec : }
\begin{align}
\begin{cases}
\alpha_{i} = 1 & \text{ si $i$ est l'indice d'un site offshore}\\
\alpha_{i} = 0 & \text{ si $i$ est l'indice d'un site onshore}
\end{cases}
\end{align}
$

 - $ 
\sum_{i=1}^{\frac{N_{heures}}{T}-1} \frac{\sum_{j=1}^{N}p_{j}^{installée}|\sum_{k=Ti+1}^{Ti+T} \eta_{jk}-\sum_{k=Ti+T+1}^{Ti+2T}\eta_{jk}|}{\frac{N_{heures}}{T}-1} \leq \delta P
$ 

    $\begin{align}
\Leftrightarrow
\begin{cases}
    \sum_{j=1}^{N}p_{j}^{installée}\sum_{k=Ti+1}^{Ti+T} \eta_{jk}-\sum_{k=Ti+T+1}^{Ti+2T}\eta_{jk} - l & \leq 0\\
    \sum_{j=1}^{N}p_{j}^{installée}(-\sum_{k=Ti+1}^{Ti+T} \eta_{jk}+\sum_{k=Ti+T+1}^{Ti+2T}\eta_{jk}) - l & \leq 0\\
    \sum_{i=1}^{\frac{N_{heures}}{T}-1} l_{i} & \leq \delta P n_{interval}\\
\end{cases}
\end{align}
$


 - $ p^{installée} \geq 0 $

Nous résolvons par la méthode du simplex et imprimons les valeurs de nos variables ainsi que de notre fonction objectif à l'optimum. 

In [10]:
#TODO contraince active, variable =0
#TODO signature de la fonction
def attribution_puissance(P = 500000, kappa = 0.17, delta = 0.02, T = 3):
    s = CyClpSimplex()
    
    # Add variables
    x = s.addVariable('x', N_sites)
    
    # Add constraints
    p_max = CyLPArray(sites["capacites"])
    s += 0 <= x <= p_max
    
    s.addConstraint(x.sum() == P,"contrainte puissance max")
    
    offshorearray = np.zeros(N_sites)
    offshorearray[indice_offshore] = 1             # set à 1 les sites offshore
    s.addConstraint((CyLPArray(offshorearray)*x) == kappa*P,"contrainte offshore")
    
    n_interval = N_heures//T  # nombre d'intervalle sur les heures imparties 
    L = s.addVariable('l', n_interval-1) # vecteur L de longeur n_interval pour gérer valeurs absolues 
    
    sum = np.sum(rend_sites_matrix.copy().reshape(N_sites, n_interval, T), axis = 2)
    sum_shifted = sum.copy()
    sum_shifted = np.roll(sum, 1, axis = 1)
    sum = np.delete(sum, np.s_[0],  1)
    sum_shifted = np.delete(sum_shifted, np.s_[0],  1)
    diff = CyLPArray((sum_shifted - sum).transpose())
    
    #print ("diff.shape : ",diff.shape)
    s += diff*x - L <= 0
    s += -diff*x - L <= 0
    #print("A droite : ",delta*P*(n_interval-1)*T)
    s.addConstraint(L.sum() <= delta*P*(n_interval-1)*T,"variabilité")  # contrainte de variabilité
    
    #print ("\nConstraints :\n", s.constraints)
    #print ("\nConstraintMatrix :\n", s.coefMatrix)
    
    # Set the objective function
    c = -CyLPArray(np.sum(rend_sites_matrix, axis = 1))
    
    s.objective = c * x
    #print("\nc = ",s.objective)
    
    # Solve using primal Simplex
    s.primal()
    
    dual_variable = s.dualVariableSolution["x"]

    dual_constraint = s.dualConstraintSolution
    
    # Return values
    return s.primalVariableSolution['x'], -s.objectiveValue, s.getStatusString(), s.primalVariableSolution['l'],dual_constraint,dual_variable
    
## Solutions ##

PUISSANCE_INSTALLEE, ENERGIE_TOTALE, type_sol, l_sol,dual_constraint,dual_variable = attribution_puissance(500000,0.17,0.02,3)

print ("\nlsol = ",l_sol) 
print ("\nPuissance installée = ",PUISSANCE_INSTALLEE) 
print("\nEnergie totale = ", ENERGIE_TOTALE, "MWh") #192644543.2528026
print("\nEtat de la solution ? = ", type_sol)


lsol =  [1.70313518e+04 4.28630154e+04 3.84177033e+04 6.71243454e+04
 3.35007241e+04 4.45884113e+04 4.69673418e+04 4.29886975e+04
 2.99179341e+04 4.56202721e+03 7.74742805e+04 7.07993558e+04
 4.65478283e+04 4.70841124e+04 3.53252986e+04 1.05374991e+04
 2.17534944e+04 2.86676385e+04 1.41179284e+04 4.14028945e+04
 9.31690838e+04 4.67097377e+04 4.10586077e+04 1.10357728e+04
 3.11230113e+04 3.08118159e+04 8.91723535e+04 1.06520962e+05
 1.03367848e+05 3.39799954e+04 3.66874639e+04 2.21255689e+04
 3.71491001e+04 4.13290653e+03 4.08864511e+04 2.11499056e+04
 2.41277359e+04 1.43714974e+04 4.75632587e+04 3.61635903e+04
 5.92440558e+04 3.07921670e+04 2.65745984e+04 5.45661286e+03
 5.97895965e+04 4.78119019e+03 5.37470870e+04 4.77472840e+04
 1.03046813e+02 1.73526902e+04 2.28376404e+04 4.45851074e+03
 5.64835260e+04 1.23782436e+04 5.27557378e+04 3.15327378e+04
 4.66107398e+04 2.26282802e+04 7.38762017e+03 1.37244067e+04
 1.31662857e+04 4.22604236e+04 1.07215460e+05 7.57554633e+04
 2.33963331e+04

In [11]:
print("Vérifications : ")
print("Somme des puissances installées = ", PUISSANCE_INSTALLEE.sum())
print("Somme des puissances installées (offshore) = ", PUISSANCE_INSTALLEE[indice_offshore].sum())


T=3
n_interval = N_heures//T
P = 500000
delta = 0.02

print("Vérifications : ")
print("Somme des puissances installées = ", PUISSANCE_INSTALLEE.sum())
print("Somme des puissances installées (offshore) = ", PUISSANCE_INSTALLEE[indice_offshore].sum())

var = np.sum(l_sol)
print("Variabilité obtenue : ",var," != ", delta*P*T*(n_interval-1), " : Variabilité théorique")

Vérifications : 
Somme des puissances installées =  499999.99999999953
Somme des puissances installées (offshore) =  85000.0
Vérifications : 
Somme des puissances installées =  499999.99999999953
Somme des puissances installées (offshore) =  85000.0
Variabilité obtenue :  7290000.0  !=  7290000.0  : Variabilité théorique


# Comparaison Bruno

In [12]:
### Comparaison Bruno ###

x = np.array([8364.15, 7509.6070033422275, 5003.709632088747, 4225.781587884473, 4225.781587884473, 5003.709632088747, 5459.416856470489, 5459.416856470489, 4985.224272064392, 4190.586795328975, 3553.7715750074103, 0.0, 0.0, 0.0, 0.0, 3601.871865159856, 3509.475537079193, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2824.057294957548, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1041.0659974681794, 0.0, 0.0, 0.0, 0.0, 1023.9941305077906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4790.162329484013, 4341.940871528012, 3617.42590533175, 2859.4708936562224, 0.0, 15478.589743589739, 12382.871794871797, 2981.220960539032, 0.0, 0.0, 2070.4135366810383, 15478.589743589742, 12382.871794871797, 2934.4207221592337, 1936.0260798300565, 2934.420722159234, 0.0, 2934.4207221592324, 0.0, 0.0, 2890.044546691392, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2812.8480008256297, 0.0, 0.0, 0.0, 3193.171206607684, 2812.8480008256297, 2273.160183003077, 1890.9163071804276, 1565.1390758495074, 0.0, 0.0, 0.0, 0.0, 0.0, 1852.6817040508786, 1662.284485052644, 1861.015318354765, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3780.556303017137, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2775.653251688818, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1828.3664386318203, 2749.681775776317, 0.0, 0.0, 0.0, 0.0, 2411.014537578884, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3193.171206607672, 2784.688009560378, 2229.3188424956647, 2215.6221041827766, 4560.716810433932, 0.0, 0.0, 0.0, 5945.5844090692235, 3922.6754246958753, 3075.6529221632186, 2924.969337202257, 0.0, 5873.124944334716, 4560.716810433932, 4560.716810433932, 4560.716810433932, 4103.82389903871, 0.0, 0.0, 0.0, 0.0, 2130.8188049872338, 0.0, 0.0, 5804.013257592906, 4560.716810433932, 444.5642482349023, 3339.721399609869, 3008.950955081254, 2918.1418609933025, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5738.057252460194, 4059.7928565964644, 3315.413538094309, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5675.080859583411, 3744.201310787703, 467.3351549338296, 0.0, 0.0, 0.0, 0.0, 0.0, 4045.488016243408, 0.0, 0.0, 0.0, 2901.1557548649243, 649.7806270167207, 0.0, 2901.155754864924, 0.0, 0.0, 0.0, 0.0, 4017.511850293067, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5502.476119078712, 5502.476119078709, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 423.7862236583094, 278.25644946736975, 303.5983424980732, 278.25644946736975, 278.25644946736975, 195.53438764454938, 231.1302973752861, 276.20599265520445, 411.7405978479937, 0.0, 210.10671946513256, 267.9689206209433, 406.15916983239856, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2256.341615939375, 2256.341615939375, 0.0, 0.0, 0.0, 3090.3370844529577, 2256.341615939374, 2561.419983651261, 2561.4199836512607, 2561.419983651261, 1799.6113098498593, 1689.9069656198046, 0.0, 1627.4847106991094, 1799.6113098498595, 1568.7471026977546, 1324.9760881891837, 1568.7471026977544, 0.0, 3032.5941947078845, 1481.49291586401, 1788.2604669873108, 2561.4199836512507, 1471.3683532128484, 1621.9318830329885, 2561.4199836512507, 2212.4615212276885, 1756.367955599125, 1449.9867929546804, 2931.384731611063, 1934.0086759673081, 2212.461521227688, 0.0, 2931.3847316110628, 2233.752922850511, 0.0, 0.0, 3032.5941947078827, 3005.59029744352, 2979.7549277872386, 2561.4199836512503, 0.0, 0.0, 0.0, 0.0, 1864.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 331.4248575090612, 0.0, 0.0, 0.0, 321.76528137850414, 212.2901998758525, 169.03135466185503, 0.0, 0.0, 163.76140132119062, 151.34641173949302, 137.25485337849676, 774.8622441683791, 163.76140132119062, 169.03135466185503, 163.7614013211906, 0.0, 0.0, 312.87629310671434, 206.42548239640166, 169.03135466185506, 0.0, 0.0, 692.4215297804024, 122.05554230373993, 111.51910482248823, 110.56118566958746, 0.0, 0.0, 304.6666133563502, 201.00895119721943, 162.762569219941, 0.0, 297.05937806306116, 195.98989379348737, 0.0, 0.0, 289.9893525132177, 191.32527038450147, 0.0, 283.4007746647698, 186.97829094371616, 146.60541609126298, 0.0, 277.2456666859685, 0.0, 0.0, 0.0, 0.0, 271.4824993695196, 179.11490158157451, 0.0, 141.79874823077841, 124.23487111324378, 169.031354661855, 169.03135466185503, 169.03135466185503, 159.51214667173642, 140.48778435688766, 0.0, 108.30730666275021, 0.0, 0.0, 260.9919213653762, 172.19346189478298, 135.01267855722327, 0.0, 0.0, 0.0, 0.0, 0.0, 256.2050878654093, 169.0352165951519, 132.53630024083014, 0.0, 0.0, 0.0, 0.0, 0.0, 251.69008095455214, 0.0, 0.0, 0.0, 169.03135466185506, 1170.3296703296708, 0.0, 0.0, 0.0, 0.0, 0.0, 1462.9120879120876, 0.0, 0.0, 1462.9120879120876, 1462.9120879120876, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2677.632177721905, 1974.3612731528406, 0.0, 2641.3350252664336, 0.0, 0.0, 0.0, 0.0, 2028.8891905928795, 1265.1726756941173, 0.0, 0.0, 1072.521495356328, 0.0, 0.0, 1129.4383615059805, 1263.4507257163655, 1257.511436585886, 0.0, 1399.6548271591882, 1697.1857323552897, 0.0, 429.5095356348148, 797.8225253865038, 526.3755864756439, 783.762795949554, 0.0, 0.0, 783.7627959495543, 0.0, 0.0, 0.0, 508.336750734642, 526.3635604171395, 770.4817741124199, 0.0, 0.0, 0.0, 757.9191554625528, 500.0481965083975, 487.80689567952203, 526.3635604171395, 757.9191554625529, 0.0, 0.0, 746.0210782593623, 0.0, 492.1980862206743, 746.0210782593626, 0.0, 734.7392732947403, 0.0, 734.73927329474, 526.3635604171394, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 724.0303467444272, 0.0, 0.0, 724.0303467444278, 0.0, 0.0, 724.0303467444277, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 713.8551721603258, 0.0, 0.0, 704.1783725496058, 464.59105797019515, 364.27247927394126, 364.27247927394126, 0.0, 704.1783725496057, 0.0, 0.0, 694.9678772190953, 526.3635604171395, 0.0, 526.3635604171395, 526.3635604171395, 477.1110661931087, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 617.0834560509142, 0.0, 0.0, 0.0, 682.3657927951205, 613.5330509164743, 556.5934740706066, 529.1743457865794, 504.313260711241, 0.0, 602.6255834461707, 545.4748895537715, 545.3648374949795, 564.0662717492205, 0.0, 0.0, 768.0398494973706, 663.0271904857381, 592.1654260312632, 582.1391455148799, 653.7616002110583, 0.0, 0.0, 0.0, 0.0, 0.0, 653.7616002110583, 582.1391455148799, 572.5318674156151, 644.7710799616351, 0.0, 0.0, 0.0, 0.0, 732.7869013713182, 0.0, 572.5318674156151, 636.0567781446618, 0.0, 0.0, 0.0, 0.0, 0.0, 563.3278552624148, 554.5109509763961, 627.6178013517597, 719.2683236679289, 0.0, 1389.9341008229108, 0.0, 518.724236261854, 593.0063364296295, 704.7004084866473, 0.0, 1362.2494685456347, 0.0, 0.0, 0.0, 0.0, 0.0, 1336.2245168196562, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1311.716921921209, 0.0, 169.03135466185506, 0.0, 693.538373972414, 596.5470897597592, 0.0, 0.0, 865.4054846939507, 1288.6010227917045, 0.0, 0.0, 1266.765413329895, 0.0, 655.3034812854587, 835.7672248599972, 0.0, 0.0, 0.0, 802.0136550828792, 687.4160686626418, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 865.4054846939506, 0.0, 865.4054846939505, 1226.549064892153, 0.0, 0.0, 9287.153846153846, 0.0, 0.0, 0.0, 0.0, 5449.623686806249, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

print("sol = ", np.dot(x,np.sum(rend_sites_matrix, axis = 1)))

print(np.array(sites["capacites"])[160])
for i in range(len(x)):
    if x[i] != PUISSANCE_INSTALLEE[i]:
        print("At ", i, " : ", x[i], " != ", PUISSANCE_INSTALLEE[i])

offshorearray = np.zeros(N_sites)
offshorearray[indice_offshore] = 1

print("\nContrainte tot : ", np.dot(np.ones(N_sites), x))
print("\nContrainte offshore : ", np.dot(offshorearray, x))
print("\nContrainte capacité max : ")
for i in range(len(x)):
    if x[i] > np.array(sites["capacites"])[i]:
        print(x[i], " >= ", np.array(sites["capacites"])[i])

sum = np.sum(rend_sites_matrix.copy().reshape(N_sites, n_interval, T), axis = 2)
sum_shifted = sum.copy()
sum_shifted = np.roll(sum, 1, axis = 1)
diff = np.asmatrix(sum_shifted - sum).transpose()
print("Variabilité : ", np.dot(diff,x))


sol =  192506257.5725679
5626.3
At  7  :  5459.416856470489  !=  5459.41685647049
At  8  :  4985.224272064392  !=  4985.224272064391
At  10  :  3553.7715750074103  !=  3553.771575007411
At  15  :  3601.871865159856  !=  3601.8718651598556
At  50  :  1041.0659974681794  !=  1041.0659974681796
At  55  :  1023.9941305077906  !=  0.0
At  67  :  15478.589743589739  !=  15478.58974358974
At  69  :  2981.220960539032  !=  2981.2209605390317
At  73  :  15478.589743589742  !=  15478.58974358974
At  75  :  2934.4207221592337  !=  2934.4207221592346
At  76  :  1936.0260798300565  !=  1936.026079830056
At  79  :  2934.4207221592324  !=  2934.420722159233
At  84  :  0.0  !=  2070.4135366810383
At  93  :  3193.171206607684  !=  3193.171206607685
At  95  :  2273.160183003077  !=  2273.1601830030772
At  97  :  1565.1390758495074  !=  0.0
At  103  :  1852.6817040508786  !=  507.8708344835125
At  104  :  1662.284485052644  !=  1662.2844850526437
At  141  :  2749.681775776317  !=  2749.6817757763165
At  

In [13]:
rend_moyen = ENERGIE_TOTALE/(N_heures*500000)
print("Rendement moyen sur une année :", rend_moyen )

Rendement moyen sur une année : 0.5260233795107111


In [64]:
#TODO faire un dataframe panda pour classer les 10 meilleurs sites selon leurs rendements moyen sur l'année
#TODO variabilité des best sites (Est -ce que les best sites sont compatible savec la notion de variabilité 
#     ou alor est-ce qu'ils sont tres tres bons sur toutes l'année mais produisent beaucoup de maniere tres instable )

dict = {}
for i in range(len(rend_sites_matrix)):
    dict[i] = rend_sites_matrix[i].sum()

dict_sorted = sorted(dict.items(), key=lambda x: x[1],reverse=True)
dict_best = dict_sorted[:10]

index_best = np.zeros(10)
rendement_best = np.zeros(10)
for i in range(10):
    index_best[i] = dict_best[i][0]
    rendement_best[i]=(dict_best[i][1])/N_heures

classement = pd.DataFrame([],columns = sites.columns)

for i in range(10):
    ligne = sites[sites.index==index_best[i]]
    classement= classement.append(ligne)
del classement["couleur"]
del classement["scores"]
del classement["capacites"]
classement.index.name = "index site"
classement["rendement moyen"] = rendement_best
classement["différence"] = rendement_best - rend_moyen


In [65]:
# Classement des meilleurs sites suivant leur rendement moyen
print("Classement des meilleurs sites suivant leur rendement moyen")
classement.head(10)

Classement des meilleurs sites suivant leur rendement moyen


,latitude,longitude,pays,capacite offshore,rendement moyen,différence
index site,,,,,,
67,57.0,6.0,Danemark,Oui,0.705639,0.179615
68,57.0,7.0,Danemark,Oui,0.693775,0.167751
69,57.0,8.0,Danemark,Non,0.685381,0.159358
2,54.0,9.0,Allemagne,Non,0.671455,0.145431
408,57.0,5.0,Norvège,Oui,0.662672,0.136649
75,56.0,8.0,Danemark,Non,0.661150,0.135127
399,58.0,6.0,Norvège,Oui,0.657555,0.131531
73,56.0,6.0,Danemark,Oui,0.655823,0.129800
74,56.0,7.0,Danemark,Oui,0.655724,0.129701


# Question I.3. 

### a)

Dans ce point, nous allons étudier l'impact d'une faible variation $\Delta P$ de la contrainte :
$$ \sum_{i=1}^{N_{sites}}p_{i}^{installée} = P$$
qui deviendra : 
$$ \sum_{i=1}^{N_{sites}}p_{i}^{installée} = P + \Delta P$$
sur le résultat $z$ de notre maximisation : 
$$ \text{max } \sum_{j=1}^{N_{heures}}\sum_{i=1}^{N_{sites}} \eta_{ij}p_{i}^{installée}$$

Notre modélisation pouvant se rapporter à un modèle du type:
$$
\text{min } c^{T}x \\
Ax \geq b \\
x \geq 0
$$

Nous pouvons facilement conclure que cette variation $\Delta P$ correspond en fait à une variation $\Delta b$ de notre vecteur contrainte.

Nous pouvons observer que sous la condition $\Delta b << b$ il est possible de réécrire le résultat de notre maximisation comme étant :
$$ z + \Delta z\\
\text{Avec}\\
\Delta z = y_{*}^{T} \Delta b\\
\text{Où $y_{*}$ est la solution de notre problème dual.}
$$
\
notre vecteur $b$ étant de la forme :
$$ b =
\begin{pmatrix}
0\\
.\\
.\\
0\\
\Delta P \\
0\\
.\\
.\\
0
\end{pmatrix}
$$
Il suffit que notre programme aille chercher la composante appropriée de $y_{*}$ et la multiplie par $\Delta P$ pour obtenir notre variation $\Delta z$

### b) 
TODO : A rédiger, mais les dix meilleurs sites sont bien ceux avec les meilleurs rendements moyen sur l'année

Notre modélisation pouvant se rapporter à un modèle du type:
$$
\text{min } c^{T}x \\
Ax \geq b \\
x \geq 0
$$

Le problème dual associé peut s'écrire sous la forme :
$$
\text{max } b^{T}y \\
A^{T}y \leq c\\
y \geq 0\\
$$

Ainsi, nous pouvons déduire que les plus grandes composantes de notre vecteur contrainte $b$ auront un impact plus marqué sur le résultat du dual. Nous cherchons donc dans notre programme les dix plus grandes composantes de $b$ qui s'avèrent également être les dix meilleurs sites du point de vue de la rentabilité.

$\it{\text{( Voir "Analyse du dual" pour les résultats du a) et b) )}}$

### Analyse du dual

In [66]:
result = {}
for i in range(len(dual_variable)):
    result[i]=dual_variable[i]
result_sorted = sorted(dict.items(), key=lambda x: x[1],reverse=True)

array_capacite = np.zeros((10,2))
                          
for i in range(10):
    array_capacite[i][0] = result_sorted[i][0]
    array_capacite[i][1] = result_sorted[i][1]
array_capacite_df = pd.DataFrame (array_capacite,columns = ["index site","coûts réduits du probèleme dual"])

print("I.3.a) Solution du problème dual associée à la contrainte de la puissance P: ", dual_constraint["contrainte puissance max"][0])

print("I.3.b) Classement des dix sites pour lesquels une augmentation de leur capacité maximale serait la plus rentable \n       en termes d'énergie produite")
array_capacite_df.head(10)

I.3.a) Solution du problème dual associée à la contrainte de la puissance P:  -287.1556352446998
I.3.b) Classement des dix sites pour lesquels une augmentation de leur capacité maximale serait la plus rentable 
       en termes d'énergie produite


,index site,coûts réduits du probèleme dual
0,67.0,516.527424
1,68.0,507.843096
2,69.0,501.699114
3,2.0,491.504852
4,408.0,485.076260
5,75.0,483.962082
6,399.0,481.329901
7,73.0,480.062782
8,74.0,479.990026
9,0.0,478.607317


# Question I.4.

In [ ]:
steps = 7
coef = np.linspace(0.5,2,steps)
puiss = np.zeros(steps)
for i in range (steps):
    puiss[i] = attribution_puissance(500000*coef[i])[1]

plt.plot(coef, puiss)
plt.title("Variation de l'énergie totale produite en fonction \n de la puissance maximale installable (coef * 500000)", fontsize=11)
plt.xlabel("coef [-]", fontsize=11)
plt.ylabel("Energie totale [MWh]", fontsize=11)

plt.show()

# Question I.5. a)

Admettons que nous n'imposions plus la contrainte :

$ \sum_{i=1}^{N_{sites}} p_{i}^{installées} = P $

Mais que nous la remplacions par :

$ \sum_{i=1}^{N_{sites}} p_{i}^{installées} \leq P $

Nous pourrions nous attendre à obtenir une solution au moins aussi bonne que dans notre cas d'origine. En effet, la contrainte d'égalité stricte étant plus forte, nous procéderions en fait à une relaxation de notre problème, or, la solution d'un problème relaxé est toujours au moins aussi bonne que celle de son problème d'origine.

Aussi, c'est ce que vérifie notre bout de code ci-dessous.

In [21]:
def attribution_puissance_variante1(P = 500000, kappa = 0.17, delta = 0.02, T = 3):
    s = CyClpSimplex()
    
    # Add variables
    x = s.addVariable('x', N_sites)
    
    # Add constraints
    p_max = CyLPArray(sites["capacites"])
    s += 0 <= x <= p_max
    
    s.addConstraint(x.sum() <= P,"contrainte puissance max")
    
    offshorearray = np.zeros(N_sites)
    offshorearray[indice_offshore] = 1             # set à 1 les sites offshore
    s.addConstraint((CyLPArray(offshorearray)*x) == kappa*P,"contrainte offshore")
    
    n_interval = N_heures//T  # nombre d'intervalle sur les heures imparties
    L = s.addVariable('l', n_interval-1) # vecteur L de longeur n_interval pour gérer valeurs absolues 
    
    sum = np.sum(rend_sites_matrix.copy().reshape(N_sites, n_interval, T), axis = 2)
    sum_shifted = sum.copy()
    sum_shifted = np.roll(sum, 1, axis = 1)
    sum = np.delete(sum, np.s_[0],  1)
    sum_shifted = np.delete(sum_shifted, np.s_[0],  1)
    diff = CyLPArray((sum_shifted - sum).transpose())
    
    s += diff*x - L <= 0
    s += -diff*x - L <= 0
    s.addConstraint(L.sum() <= delta*P*(n_interval-1)*T,"variabilité")  # contrainte de variabilité
    
    #print ("\nConstraints :\n", s.constraints)
    #print ("\nConstraintMatrix :\n", s.coefMatrix)
    
    # Set the objective function
    c = -CyLPArray(np.sum(rend_sites_matrix, axis = 1))
    
    s.objective = c * x
    #print("\nc = ",s.objective)
    
    # Solve using primal Simplex
    s.primal()
    
    dual_variable = s.dualVariableSolution["x"]

    dual_constraint = s.dualConstraintSolution
    
    # Return values
    return s.primalVariableSolution['x'], -s.objectiveValue, s.getStatusString(), s.primalVariableSolution['l'],dual_constraint,dual_variable
    
## Solutions ##

PUISSANCE_INSTALLEE1, ENERGIE_TOTALE1, type_sol1, l_sol1,dual_constraint1,dual_variable1 = attribution_puissance_variante1(500000,0.17,0.02,3)

print("\nEnergie totale_variante = ", ENERGIE_TOTALE1, "MWh") #192644543.2528026
#print("\nEtat de la solution ? = ", type_sol)

print("\nEnergie totale = ", ENERGIE_TOTALE, "MWh")

print("\nAugmentation de: ", (ENERGIE_TOTALE1/ENERGIE_TOTALE)*100 -100 ,"% par rapport au problème initial")


Energie totale_variante =  192644543.84691712 MWh

Energie totale =  192524556.90092024 MWh

Augmentation de:  0.06232293060600114 % par rapport au problème initial


# Question I.5. b)

Considérons maintenant que nous n'imposons plus sur notre problème initial la contrainte :

$
\sum_{i=1}^{\frac{N_{heures}}{T}-1} \frac{\sum_{j=1}^{N}p_{j}^{installée}|\sum_{k=Ti+1}^{Ti+T} \eta_{jk}-\sum_{k=Ti+T+1}^{Ti+2T}\eta_{jk}|}{\frac{N_{heures}}{T}-1} \leq \delta P
$

Mais que nous la remplaçons par la nouvelle contrainte :

$
\text{max } ( \sum_{j=1}^{N}p_{j}^{installée}|\sum_{k=Ti+1}^{Ti+T} \eta_{jk}-\sum_{k=Ti+T+1}^{Ti+2T}\eta_{jk}| , i \in [1,\frac{N_{heures}}{T}-1] ) \leq \delta P
$

A l'inverse du point précédent, cette nouvelle contrainte est plus forte que celle d'origine, le problème est donc plus contraint que précédemment et nous nous attendons à ce que notre solution s'en trouve diminuée.

Aussi, c'est ce que confirme notre bout de code ci-dessous.

In [22]:
def attribution_puissance_variante2(P = 500000, kappa = 0.17, delta = 0.02, T = 3):
    s = CyClpSimplex()
    
    # Add variables
    x = s.addVariable('x', N_sites)
    
    # Add constraints
    p_max = CyLPArray(sites["capacites"])
    s += 0 <= x <= p_max
    
    s.addConstraint(x.sum() == P,"contrainte puissance max")
    
    offshorearray = np.zeros(N_sites)
    offshorearray[indice_offshore] = 1             # set à 1 les sites offshore
    s.addConstraint((CyLPArray(offshorearray)*x) == kappa*P,"contrainte offshore")
    
    n_interval = N_heures//T  # nombre d'intervalle sur les heures imparties
    L = s.addVariable('l', n_interval-1) # vecteur L de longeur n_interval pour gérer valeurs absolues 
    
    sum = np.sum(rend_sites_matrix.copy().reshape(N_sites, n_interval, T), axis = 2)
    sum_shifted = sum.copy()
    sum_shifted = np.roll(sum, 1, axis = 1)
    sum = np.delete(sum, np.s_[0],  1)
    sum_shifted = np.delete(sum_shifted, np.s_[0],  1)
    diff = CyLPArray((sum_shifted - sum).transpose())
    s += diff*x - L <= 0
    s += -diff*x - L <= 0
    s.addConstraint(np.array(L).max() <= delta*P,"variabilité")  # contrainte de variabilité
    
    #print ("\nConstraints :\n", s.constraints)
    #print ("\nConstraintMatrix :\n", s.coefMatrix)
    
    # Set the objective function
    c = -CyLPArray(np.sum(rend_sites_matrix, axis = 1))
    
    s.objective = c * x
    #print("\nc = ",s.objective)
    
    # Solve using primal Simplex
    s.primal()
    
    dual_variable = s.dualVariableSolution["x"]

    dual_constraint = s.dualConstraintSolution
    
    
    # Return values
    return s.primalVariableSolution['x'], -s.objectiveValue, s.getStatusString(), s.primalVariableSolution['l'],dual_constraint,dual_variable
    
## Solutions ##

PUISSANCE_INSTALLEE2, ENERGIE_TOTALE2, type_sol2, l_sol2,dual_constraint2,dual_variable2 = attribution_puissance_variante2(500000,0.17,0.02,3)

#print ("\nlsol = ",l_sol2) 
#print ("\nPuissance installée2 = ",PUISSANCE_INSTALLEE2) 
print("\nEnergie totale2 = ", ENERGIE_TOTALE2, "MWh") #192644543.2528026
#print("\nEtat de la solution2 ? = ", type_sol2)

print("\nEnergie totale = ", ENERGIE_TOTALE, "MWh")

print("\nDiminution de: ", (ENERGIE_TOTALE2/ENERGIE_TOTALE)*100 -100 ,"% par rapport au problème initial")


Energie totale2 =  84229021.81038766 MWh

Energie totale =  192524556.90092024 MWh

Diminution de:  -56.25024507718524 % par rapport au problème initial


In [23]:
"""## Visualisation des rendements pour un pays

pays_test = 'France'
index_test = liste_pays_index_dic[pays_test]

# nombre de sites éoliens
index_sites_eoliens = sites[sites.pays==pays_test]['index site']
n_eoliens = len(index_sites_eoliens)
print('il y a '+str(n_eoliens)+' sites éoliens en '+str(pays_test))

# rendements eolien au hasard
random_site = np.random.choice(index_sites_eoliens)
plt.figure(figsize=(9,6))
plt.grid()
plt.title('rendement éolien: '+str(pays_test))
if sites.loc[random_site]['capacite offshore']=='Oui':
    plt.plot(vecteur_temps,matrice_rendements_offshore[random_site])
    plt.legend(['offshore'])
else:
    plt.plot(vecteur_temps,matrice_rendements_onshore[random_site])
    plt.legend(['onshore'])
plt.xlabel('temps : [années]')
plt.ylabel('rendement éolien: [/]')

print('rendement moyen offshore pour le pays: '+str(np.mean(matrice_rendements_offshore[index_sites_eoliens])))
print('rendement moyen onshore pour le pays: '+str(np.mean(matrice_rendements_onshore[index_sites_eoliens])))

# capacité éoliennes maximales tolérables
buf_ok_offshore = (sites.pays==pays_test) & (sites['capacite offshore']=='Oui')
buf_ok_onshore = (sites.pays==pays_test) & (sites['capacite offshore']=='Non')

print(' => capacité éolienne totale offshore : [MW] '+str(sites[buf_ok_offshore]['capacites'].sum()))
print(' => capacité éolienne totale offshore : [MW] '+str(sites[buf_ok_onshore]['capacites'].sum()))

# latitude/longitude
print('le pays '+str(pays_test)+' se trouve  à une latitude de '+str(sites.loc[random_site]['latitude'])+' et une longitude de '+\
      str(sites.loc[random_site]['longitude']))"""

"## Visualisation des rendements pour un pays\n\npays_test = 'France'\nindex_test = liste_pays_index_dic[pays_test]\n\n# nombre de sites éoliens\nindex_sites_eoliens = sites[sites.pays==pays_test]['index site']\nn_eoliens = len(index_sites_eoliens)\nprint('il y a '+str(n_eoliens)+' sites éoliens en '+str(pays_test))\n\n# rendements eolien au hasard\nrandom_site = np.random.choice(index_sites_eoliens)\nplt.figure(figsize=(9,6))\nplt.grid()\nplt.title('rendement éolien: '+str(pays_test))\nif sites.loc[random_site]['capacite offshore']=='Oui':\n    plt.plot(vecteur_temps,matrice_rendements_offshore[random_site])\n    plt.legend(['offshore'])\nelse:\n    plt.plot(vecteur_temps,matrice_rendements_onshore[random_site])\n    plt.legend(['onshore'])\nplt.xlabel('temps : [années]')\nplt.ylabel('rendement éolien: [/]')\n\nprint('rendement moyen offshore pour le pays: '+str(np.mean(matrice_rendements_offshore[index_sites_eoliens])))\nprint('rendement moyen onshore pour le pays: '+str(np.mean(matr

In [24]:
"""indice_offshore= offshore.index
rend_offshore_matrix = rend_offshore_brut[indice_offshore]
rend_offshore_array = np.array([np.sum(rend_offshore_matrix[i]) for i in range(len(indice_offshore))])


indice_onshore= onshore.index
rend_onshore_matrix = rend_onshore_brut[indice_onshore]
rend_onshore_array = np.array([np.sum(rend_onshore_matrix[i]) for i in range(len(indice_onshore))])

maxoff= max(rend_offshore_array)
maxon = max(rend_onshore_array)

for i in range(len(rend_offshore_array)):
    if (rend_offshore_array[i] == max(rend_offshore_array) ) :
        print("Max offshore = {}  at  index = {} in offshore submatrix".format(maxoff,i))

for i in range(len(rend_onshore_array)):
    if (rend_onshore_array[i] == max(rend_onshore_array) ) : 
        print("Max onshore = {}  at  index = {} in onshore submatrix\n".format(maxon,i))
        
for i in range(642):
    if (rend_offshore_brut[i].sum() == maxoff):
        print("Max offshore = {}  at  index = {} in total matrix".format(maxoff,i))
    elif (rend_onshore_brut[i].sum() == maxon):
        print("Max onshore = {}  at  index = {} in total matrix".format(maxon,i))"""

'indice_offshore= offshore.index\nrend_offshore_matrix = rend_offshore_brut[indice_offshore]\nrend_offshore_array = np.array([np.sum(rend_offshore_matrix[i]) for i in range(len(indice_offshore))])\n\n\nindice_onshore= onshore.index\nrend_onshore_matrix = rend_onshore_brut[indice_onshore]\nrend_onshore_array = np.array([np.sum(rend_onshore_matrix[i]) for i in range(len(indice_onshore))])\n\nmaxoff= max(rend_offshore_array)\nmaxon = max(rend_onshore_array)\n\nfor i in range(len(rend_offshore_array)):\n    if (rend_offshore_array[i] == max(rend_offshore_array) ) :\n        print("Max offshore = {}  at  index = {} in offshore submatrix".format(maxoff,i))\n\nfor i in range(len(rend_onshore_array)):\n    if (rend_onshore_array[i] == max(rend_onshore_array) ) : \n        print("Max onshore = {}  at  index = {} in onshore submatrix\n".format(maxon,i))\n        \nfor i in range(642):\n    if (rend_offshore_brut[i].sum() == maxoff):\n        print("Max offshore = {}  at  index = {} in total matr